## Overview

This notebook is designed to provide a broad overview of Hail's functionality, with emphasis on the functionality to manipulate and query a genetic dataset. We walk through a genome-wide SNP association test, and demonstrate the need to control for confounding caused by population stratification.

Each notebook starts the same: we import the `hail2` package and create a [HailContext](https://hail.is/docs/devel/api1/hail.api1.HailContext.html). This object is the entry point for most Hail functionality.


In [ ]:
from hail2 import *
hc = HailContext()

If the above cell ran without error, we're ready to go! 

Before using Hail, we import some standard Python libraries for use throughout the notebook.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from collections import Counter
from math import log, isnan
from pprint import pprint
%matplotlib inline

 Installing and importing [seaborn](http://seaborn.pydata.org/installing.html) is optional; it just makes the plots prettier.

In [ ]:
# optional
import seaborn

## Check for tutorial data or download if necessary

This cell downloads the necessary data from Google Storage if it isn't found in the current working directory.

In [ ]:
import os
if os.path.isdir('data/1kg.vds') and os.path.isfile('data/1kg_annotations.txt'):
    print('All files are present and accounted for!')
else:
    import sys
    sys.stderr.write('Downloading data (~16MB) from Google Storage...\n')
    import urllib
    import tarfile
    urllib.urlretrieve('https://storage.googleapis.com/hail-tutorial/hail-tutorial-data-v4.tar.gz',
                       'hail-tutorial-data-v4.tar.gz')
    sys.stderr.write('Download finished!\n')
    sys.stderr.write('Extracting...\n')
    tarfile.open('hail-tutorial-data-v4.tar.gz').extractall()
    if not (os.path.isfile('data/1kg.vcf.bgz') and os.path.isfile('data/1kg_annotations.txt')):
        raise RuntimeError('Something went wrong!')
    
    sys.stderr.write('Importing data...\n')
    (hc.import_vcf('data/1kg.vcf.bgz')
     .write('data/1kg.vds'))
    sys.stderr.write('Done!\n')

## Loading data from disk

Hail has its own internal data representation, called a MatrixTable. This is both an on-disk file format and a [Python object](https://hail.is/docs/devel/api2/hail.api2.MatrixTable.html?highlight=annotate_cols#matrixtable). Here, we read a MatrixTable from disk.

This dataset was created by downsampling a public 1000 genomes VCF to about 50 MB.

In [ ]:
ds = hc.read('data/1kg.vds')

## Getting to know our data

It's important to have easy ways to slice, dice, query, and summarize a dataset. Some of these methods are demonstrated below.

The [summarize](https://hail.is/hail/hail.VariantDataset.html#hail.VariantDataset.summarize) method is useful for providing a broad overview of the data contained in a dataset. 

`summarize` is not currently supported in API2, but we can convert the dataset to a Hail 0.1 object first by using the `to_hail1` method, and then calling `summarize`.

In [ ]:
ds.to_hail1().summarize().report()

The [rows_table](https://hail.is/docs/devel/api2/hail.api2.MatrixTable.html?highlight=rows_table#hail.api2.MatrixTable.rows_table) method can be used to get a table with all the row fields in our MatrixTable. 

We can use `rows_table` along with [select](https://hail.is/docs/devel/api2/hail.api2.Table.html?highlight=select#hail.api2.Table.select) to pull out 5 variants. The `select` method takes either a string refering to a field name in the table, or a Hail [Expression](https://hail.is/docs/devel/expr/expression.html#hail.expr.expression.Expression). Here, we pass the string 'v' to pull out the variants field.

Use the `show` method to display the variants.

In [ ]:
ds.rows_table().select('v').show(5)

There are often several ways to do something in Hail. Here are two ways to peek at the first few sample IDs:

In [ ]:
ds.cols_table().select('s').show(5)

In [ ]:
ds.to_hail1().sample_ids[:5]

To look at the first few genotype calls, we can use [entries_table](https://hail.is/docs/devel/api2/hail.api2.MatrixTable.html?highlight=entries_table#hail.api2.MatrixTable.entries_table) along with `select` and `take`. The `take` method collects the first n rows into a list. Alternatively, we can use the `show` method, which prints the first n rows to the console in a table format. 

Try changing `take` to `show` in the cell below.

In [ ]:
ds.entries_table().select('DP','GT','AD','GQ','PL').take(5)

## Adding column fields

A Hail MatrixTable can have any number of row fields and column fields for storing data associated with each row and column. Annotations are usually a critical part of any genetic study. Column fields are where you'll store information about sample phenotypes, ancestry, sex, and covariates. Row fields can be used to store information like gene membership and functional impact for use in QC or analysis. 

In this tutorial, we demonstrate how to take a text file and use it to annotate the columns in a MatrixTable. 

iPython supports various cell "magics". The `%%sh` magic is one which interprets the cell with bash, rather than Python. We can use this to look at the first few lines of our annotation file. This file contains the sample ID, the population and "super-population" designations, the sample sex, and two simulated phenotypes (one binary, one discrete).

In [ ]:
%%sh
head data/1kg_annotations.txt | column -t 

This file can be imported into Hail with [HailContext.import_table](https://hail.is/hail/hail.HailContext.html#hail.HailContext.import_table). This method produces a [Table](https://hail.is/docs/devel/api2/hail.api2.Table.html#hail.api2.Table) object. Think of this as a Pandas or R dataframe that isn't limited by the memory on your machine -- behind the scenes, it's distributed with Spark.

In [ ]:
table = hc.import_table('data/1kg_annotations.txt', impute=True)\
    .key_by('Sample')

A good way to peek at the structure of a `Table` is to look at its `schema`. 



In [ ]:
print(table.schema)

The Python `pprint` method makes illegible printouts pretty:

In [ ]:
pprint(table.schema)

Although we used the `%%sh` magic to look at the first lines of the table, there's a better way. We can use the `show` method:

In [ ]:
table.show(10, width=100)

Now we'll use this table to add sample annotations to our dataset, storing the annotations in column fields in our MatrixTable. First, we'll print the existing column schema:

In [ ]:
pprint(ds.col_schema)

We use the [annotate_cols](https://hail.is/docs/devel/api2/hail.api2.MatrixTable.html?highlight=annotate_cols#hail.api2.MatrixTable.annotate_cols) method to join the table with the MatrixTable.

In [ ]:
ds = ds.annotate_cols(**table[ds.s])

In [ ]:
pprint(ds.col_schema)

## Query functions and the Hail Expression Language

Hail has a number of useful query functions that can be used for gathering statistics on our dataset. These query functions take Hail Expressions as arguments.

We will start by looking at some statistics of the information in our table. The [aggregate](https://hail.is/docs/devel/api2/hail.api2.Table.html?highlight=aggregate#hail.api2.Table.aggregate) method can be used to aggregate over rows of the table.

`counter` is an aggregation function that counts the number of occurrences of each unique element. We can use this to pull out the population distribution by passing in a Hail Expression for the field that we want to count by.

In [ ]:
pprint(table.aggregate(pop_count = agg.counter(table.SuperPopulation)))

`stats` is an aggregation function that produces some useful statistics about numeric collections. We can use this to see the distribution of the CaffeineConsumption phenotype.

In [ ]:
pprint(table.aggregate(caffeine_stats = agg.stats(table.CaffeineConsumption)))

However, these metrics aren't perfectly representative of the samples in our dataset. Here's why:

In [ ]:
table.count()

In [ ]:
ds.count_cols()

Since there are fewer samples in our dataset than in the full thousand genomes cohort, we need to look at annotations on the dataset. We can use [aggregate_cols](https://hail.is/docs/devel/api2/hail.api2.MatrixTable.html?highlight=aggregate_cols#hail.api2.MatrixTable.aggregate_cols) to get the metrics for only the samples in our dataset.

In [ ]:
ds.aggregate_cols(pop_count = agg.counter(ds.SuperPopulation))

In [ ]:
pprint(ds.aggregate_cols(caffeine_stats = agg.stats(ds.CaffeineConsumption)))

The functionality demonstrated in the last few cells isn't anything especially new: it's certainly not difficult to ask these questions with Pandas or R dataframes, or even Unix tools like `awk`. But Hail can use the same interfaces and query language to analyze collections that are much larger, like the set of variants. 

Here we calculate the counts of each of the 12 possible unique SNPs (4 choices for the reference base * 3 choices for the alternate base). 

To do this, we need to get the alternate allele of each variant and then count the occurences of each unique ref/alt pair. This can be done with Hail's `counter` method.

In [ ]:
snp_counts = ds.aggregate_rows(snp_counts = agg.counter(ds.v.alt_allele()))
pprint(snp_counts)

We can list the counts in descending order using Python's Counter class.

In [ ]:
from collections import Counter
counts = Counter(snp_counts.snp_counts)
counts.most_common()

It's nice to see that we can actually uncover something biological from this small dataset: we see that these frequencies come in pairs. C/T and G/A are actually the same mutation, just viewed from from opposite strands. Likewise, T/A and A/T are the same mutation on opposite strands. There's a 30x difference between the frequency of C/T and A/T SNPs. Why?

The same Python, R, and Unix tools could do this work as well, but we're starting to hit a wall - the latest [gnomAD release](http://gnomad.broadinstitute.org/) publishes about 250 million variants, and that won't fit in memory on a single computer.

What about genotypes? Hail can query the collection of all genotypes in the dataset, and this is getting large even for our tiny dataset. Our 284 samples and 10,000 variants produce 10 million unique genotypes. The gnomAD dataset has about **5 trillion** unique genotypes.

Here we will use the `hist` aggregator to produce and plot a histogram of DP values for genotypes in our thousand genomes dataset.

In [ ]:
dp_hist = ds.aggregate_entries(dp_hist = agg.hist(ds.DP, 0, 30, 30)).dp_hist
plt.xlim(0, 31)
plt.bar(dp_hist.binEdges[1:], dp_hist.binFrequencies)
plt.show()

## Quality Control

QC is where analysts spend most of their time with sequencing datasets. QC is an iterative process, and is different for every project: there is no "push-button" solution for QC. Each time the Broad collects a new group of samples, it finds new batch effects. However, by practicing open science and discussing the QC process and decisions with others, we can establish a set of best practices as a community.

QC is entirely based on the ability to understand the properties of a dataset. Hail attempts to make this easier by providing the [sample_qc](https://hail.is/docs/devel/methods/index.html?highlight=sample_qc#hail.methods.sample_qc) method, which produces a set of useful metrics and stores them in a column field.

In [ ]:
pprint(ds.col_schema)

In [ ]:
ds = methods.sample_qc(ds)

In [ ]:
pprint(ds.col_schema)

Interoperability is a big part of Hail.

To pull out these new metrics, we need to convert to a Pandas DataFrame, which can be done by first converting to a Hail 0.1 KeyTable via the `to_hail1` method, and then using the the `to_pandas` method.

In [ ]:
df = ds.to_hail1().samples_table().to_pandas()

In [ ]:
df.head()

Plotting the QC metrics is a good place to start.

In [ ]:
plt.clf()
plt.subplot(1, 2, 1)
plt.hist(df["sa.sample_qc.callRate"], bins=np.arange(.75, 1.01, .01))
plt.xlabel("Call Rate")
plt.ylabel("Frequency")
plt.xlim(.75, 1)

plt.subplot(1, 2, 2)
plt.hist(df["sa.sample_qc.gqMean"], bins = np.arange(0, 105, 5))
plt.xlabel("Mean Sample GQ")
plt.ylabel("Frequency")
plt.xlim(0, 105)

plt.tight_layout()
plt.show()

Often, these metrics are correlated.

In [ ]:
plt.scatter(df["sa.sample_qc.dpMean"], df["sa.sample_qc.callRate"],
            alpha=0.1)
plt.xlabel('Mean DP')
plt.ylabel('Call Rate')
plt.xlim(0, 20)
plt.show()

Removing outliers from the dataset will generally improve association results. We can draw lines on the above plot to indicate outlier cuts. We'll want to remove all samples that fall in the bottom left quadrant.

In [ ]:
plt.scatter(df["sa.sample_qc.dpMean"], df["sa.sample_qc.callRate"],
            alpha=0.1)
plt.xlabel('Mean DP')
plt.ylabel('Call Rate')
plt.xlim(0, 20)
plt.axhline(0.97, c='k')
plt.axvline(4, c='k')
plt.show()

It's easy to filter when we've got the cutoff values decided:

In [ ]:
ds = ds.filter_cols((ds.sample_qc.dpMean >= 4) & (ds.sample_qc.callRate >= 0.97))
print('After filter, %d/1000 samples remain.' % ds.count_cols())

Next is genotype QC. To start, we'll print the post-sample-QC call rate. It's actually gone _up_ since the initial summary - dropping low-quality samples disproportionally removed missing genotypes.

Import the `hail.expr.functions` module.

In [ ]:
import hail.expr.functions as functions

call_rate = ds.aggregate_entries(call_rate = agg.fraction(functions.is_defined(ds.GT))).call_rate
print('pre QC call rate is %.3f' % call_rate)

It's a good idea to filter out genotypes where the reads aren't where they should be: if we find a genotype called homozygous reference with >10% alternate reads, a genotype called homozygous alternate with >10% reference reads, or a genotype called heterozygote without a ref / alt balance near 1:1, it is likely to be an error.

In [ ]:
ab = ds.AD[1]/ds.AD.sum()

filter_condition_ab = ((ds.GT.is_hom_ref() & (ab <= 0.1)) |
                        (ds.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (ds.GT.is_hom_var() & (ab >= 0.9)))

ds = ds.filter_entries(filter_condition_ab)

In [ ]:
post_qc_call_rate = ds.aggregate_entries(post_qc_call_rate = agg.fraction(functions.is_defined(ds.GT))).post_qc_call_rate
print('post QC call rate is %.3f' % post_qc_call_rate)

Variant QC is a bit more of the same: we can use the [row_schema](https://hail.is/docs/devel/api2/hail.api2.MatrixTable.html?highlight=row_schema#hail.api2.MatrixTable.row_schema) method to produce a variety of useful statistics, plot them, and filter.

In [ ]:
pprint(ds.row_schema)

The [cache](https://hail.is/docs/devel/api2/hail.api2.MatrixTable.html#hail.api2.MatrixTable.cache) is used to optimize some of the downstream operations.

In [ ]:
ds = methods.variant_qc(ds).cache()

In [ ]:
pprint(ds.row_schema)

In [ ]:
variant_df = ds.to_hail1().variants_table().to_pandas()

plt.clf()
plt.subplot(2, 2, 1)
variantgq_means = variant_df["va.variant_qc.gqMean"]
plt.hist(variantgq_means, bins = np.arange(0, 84, 2))
plt.xlabel("Variant Mean GQ")
plt.ylabel("Frequency")
plt.xlim(0, 80)

plt.subplot(2, 2, 2)
variant_mleaf = variant_df["va.variant_qc.AF"]
plt.hist(variant_mleaf, bins = np.arange(0, 1.05, .025))
plt.xlabel("Minor Allele Frequency")
plt.ylabel("Frequency")
plt.xlim(0, 1)

plt.subplot(2, 2, 3)
plt.hist(variant_df['va.variant_qc.callRate'], bins = np.arange(0, 1.05, .01))
plt.xlabel("Variant Call Rate")
plt.ylabel("Frequency")
plt.xlim(.5, 1)

plt.subplot(2, 2, 4)
plt.hist(variant_df['va.variant_qc.pHWE'], bins = np.arange(0, 1.05, .025))
plt.xlabel("Hardy-Weinberg Equilibrium p-value")
plt.ylabel("Frequency")
plt.xlim(0, 1)

plt.tight_layout()
plt.show()



These statistics actually look pretty good: we don't need to filter this dataset. Most datasets require thoughtful quality control, though. The [filter_rows](https://hail.is/docs/devel/api2/hail.api2.MatrixTable.html#hail.api2.MatrixTable.filter_rows) method can help!

## Let's do a GWAS!

First, we need to restrict to variants that are : 

 - common (we'll use a cutoff of 1%)
 - uncorrelated (not in linkage disequilibrium)
 
(Hail2 doesn't currently have a method for ld_prune, although it is available in Hail1.)

In [ ]:
common_ds = ds.filter_rows(ds.variant_qc.AF > 0.01)

In [ ]:
print('Samples: %d  Variants: %d' % (common_ds.count_cols(), common_ds.count_rows()))

These filters removed about 15% of sites (we started with a bit over 10,000). This is _NOT_ representative of most sequencing datasets! We have already downsampled the full thousand genomes dataset to include more common variants than we'd expect by chance.

In Hail, the association tests accept column fields for the sample phenotype and covariates. Since we've already got our phenotype of interest (caffeine consumption) in the dataset, we are good to go:

In [ ]:
gwas = methods.linreg(common_ds, common_ds.CaffeineConsumption, common_ds.GT.num_alt_alleles())
pprint(gwas.row_schema)

Looking at the bottom of the above printout, you can see the linear regression adds new row fields for the beta, standard error, t-statistic, and p-value.

In [ ]:
def qqplot(pvals, xMax, yMax):
    spvals = sorted(filter(lambda x: x and not(isnan(x)), pvals))
    exp = [-log(float(i) / len(spvals), 10) for i in np.arange(1, len(spvals) + 1, 1)]
    obs = [-log(p, 10) for p in spvals]
    plt.clf()
    plt.scatter(exp, obs)
    plt.plot(np.arange(0, max(xMax, yMax)), c="red")
    plt.xlabel("Expected p-value (-log10 scale)")
    plt.ylabel("Observed p-value (-log10 scale)")
    plt.xlim(0, xMax)
    plt.ylim(0, yMax)
    plt.show()

Python makes it easy to make a [Q-Q (quantile-quantile) plot](https://en.wikipedia.org/wiki/Q-Q_plot).

In [ ]:
qqplot(gwas.aggregate_rows(pvals = agg.collect(gwas.linreg.pval[0])).pvals, 5, 6)

## Confounded!

The observed p-values drift away from the expectation immediately. Either every SNP in our dataset is causally linked to caffeine consumption (unlikely), or there's a confounder.

We didn't tell you, but sample ancestry was actually used to simulate this phenotype. This leads to a [stratified](https://en.wikipedia.org/wiki/Population_stratification) distribution of the phenotype. The solution is to include ancestry as a covariate in our regression. 

The [linreg](https://hail.is/docs/devel/methods/index.html?highlight=linreg#hail.methods.linreg) method can also take column fields to use as covariates. We already annotated our samples with reported ancestry, but it is good to be skeptical of these labels due to human error. Genomes don't have that problem! Instead of using reported ancestry, we will use genetic ancestry by including computed principal components in our model.

The [pca](https://hail.is/docs/devel/methods/index.html#hail.methods.pca) method produces eigenvalues as a list and sample PCs as a Table, and can also produce variant loadings when asked. The [hwe_normalized_pca](https://hail.is/docs/devel/methods/index.html#hail.methods.hwe_normalized_pca) method does the same, using HWE-normalized genotypes for the PCA.

In [ ]:
pca_eigenvalues, pca_scores, _ = methods.hwe_normalized_pca(common_ds)

In [ ]:
pprint(pca_eigenvalues)

In [ ]:
pca_scores.show(5, width=100)

Now that we've got principal components per sample, we may as well plot them! Human history exerts a strong effect in genetic datasets. Even with a 50MB sequencing dataset, we can recover the major human populations.

In [ ]:
pca_table = pca_scores.join(common_ds.cols_table()).to_hail1().to_pandas()

colors = {'AFR': 'green', 'AMR': 'red', 'EAS': 'black', 'EUR': 'blue', 'SAS': 'cyan'}
plt.scatter(pca_table["pcaScores.PC1"], pca_table["pcaScores.PC2"], 
            c = pca_table["SuperPopulation"].map(colors), 
            alpha = .5)
plt.xlim(-0.6, 0.6)
plt.xlabel("PC1")
plt.ylabel("PC2")
legend_entries = [mpatches.Patch(color=c, label=pheno) for pheno, c in colors.items()]
plt.legend(handles=legend_entries, loc=2)
plt.show()

Now we can rerun our linear regression, controlling for the first few principal components and sample sex.

In [ ]:
cds = common_ds.annotate_cols(pca = pca_scores[common_ds.s])

linreg_results = methods.linreg(cds, cds.CaffeineConsumption, cds.GT.num_alt_alleles(),
                                covariates=[cds.pca.PC1, cds.pca.PC2, cds.pca.PC3, cds.isFemale])

pvals = linreg_results.aggregate_rows(pvals=agg.collect(linreg_results.linreg.pval[0])).pvals

In [ ]:
qqplot(pvals, 5, 6)

In [ ]:
def plDosage(pl):
    linear_scaled = pl.map(lambda x: 10 ** (-x / 10))
    pl_sum = linear_scaled.sum()
    normed = linear_scaled / pl_sum
    return 1 * normed[1] + 2 * normed[2]

In [ ]:
linreg_results = methods.linreg(cds, [cds.CaffeineConsumption], plDosage(cds.PL),
                                covariates=[cds.pca.PC1, cds.pca.PC2, cds.pca.PC3, cds.isFemale])

pvals = linreg_results.aggregate_rows(pvals=agg.collect(linreg_results.linreg.pval[0])).pvals

In [ ]:
qqplot(pvals, 5, 6)

That's more like it! We may not be publishing ten new coffee-drinking loci in _Nature_, but we shouldn't expect to find anything but the strongest signals from a dataset of 284 individuals anyway. 

## Rare variant analysis

Here we'll demonstrate how one can use the expression language to group and count by any arbitrary properties in row and column fields. Hail also implements the sequence kernel association test (SKAT).


In [ ]:
entries = ds.entries_table()
results = (entries.group_by(pop = entries.SuperPopulation, chromosome = entries.v.contig)
      .aggregate(n_het = agg.count_where(entries.GT.is_het())))


In [ ]:
results.show() 

What if we want to group by minor allele frequency bin and hair color, and calculate the mean GQ?

In [ ]:
entries = entries.annotate(maf_bin = functions.cond(entries.info.AF[0]<0.01, "< 1%", 
                              functions.cond(entries.info.AF[0]<0.05, "1%-5%", ">5%")))

results2 = (entries.group_by(af_bin = entries.maf_bin, purple_hair = entries.PurpleHair)
      .aggregate(mean_gq = agg.stats(entries.GQ).mean, mean_dp = agg.stats(entries.DP).mean))


In [ ]:
results2.show()

We've shown that it's easy to aggregate by a couple of arbitrary statistics. This specific examples may not provide especially useful pieces of information, but this same pattern can be used to detect effects of rare variation:

 - Count the number of heterozygous genotypes per gene by functional category (synonymous, missense, or loss-of-function) to estimate per-gene functional constraint
 - Count the number of singleton loss-of-function mutations per gene in cases and controls to detect genes involved in disease

## Eplilogue

Congrats! If you've made it this far, you're perfectly primed to read the [Overview](https://hail.is/hail/overview.html), look through the [Hail objects](https://hail.is/hail/types.html) representing many core concepts in genetics, and check out the many Hail functions defined in the [Python API](https://hail.is/hail/api.html). If you use Hail for your own science, we'd love to hear from you on [Gitter chat](https://gitter.im/hail-is/hail) or the [discussion forum](http://discuss.hail.is).

There's also a lot of functionality inside Hail that we didn't get to in this broad overview. Things like:

 - Flexible import and export to a variety of data and annotation formats (VCF, BGEN, PLINK, JSON, TSV, ...)
 - Simulation
 - Kinship and pruning (IBD, GRM, RRM, LD)
 - Family-based tests and utilities
 - Distributed file system utilities
 - Interoperability with Python and Spark machine learning libraries
 - More!

For reference, here's the full workflow to all tutorial endpoints combined into one cell. It may take a minute! It's doing a lot of work.

In [ ]:
table = hc.import_table('data/1kg_annotations.txt', impute=True).key_by('Sample')

ds = hc.read('data/1kg.vds')
ds = ds.annotate_cols(**table[ds.s])
ds = methods.sample_qc(ds)
ds = ds.filter_cols((ds.sample_qc.dpMean >= 4) & (ds.sample_qc.callRate >= 0.97))
ab = ds.AD[1]/ds.AD.sum()
filter_condition_ab = ((ds.GT.is_hom_ref() & (ab <= 0.1)) |
                        (ds.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (ds.GT.is_hom_var() & (ab >= 0.9)))
ds = ds.filter_entries(filter_condition_ab)
ds = methods.variant_qc(ds)
common_ds = ds.filter_rows(ds.variant_qc.AF > 0.01)

pca_eigenvalues, pca_scores, _ = methods.hwe_normalized_pca(common_ds)

cds = common_ds.annotate_cols(pca = pca_scores[common_ds.s])
linreg_results = methods.linreg(cds, [cds.CaffeineConsumption], cds.GT.num_alt_alleles(),
                                covariates=[cds.pca.PC1, cds.pca.PC2, cds.pca.PC3, cds.isFemale])
pvals = linreg_results.aggregate_rows(pvals=agg.collect(linreg_results.linreg.pval[0])).pvals